<a href="https://colab.research.google.com/github/hindu-muppala/EYHack_DataScience/blob/main/FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# red, thermal 10, swir 22, nvio08 bands

In [ ]:
!pip install rioxarray stackstac pystac_client planetary_computer  pyodc osmnx odc-stac netCDF4 rioxarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.2/35.2 MB 14.6 MB/s eta 0:00:00


In [ ]:
lower_left = (40.75, -74.01)
upper_right = (40.88, -73.86)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import rioxarray as rio
import rasterio
from matplotlib.cm import jet,RdYlGn

import stackstac
import pystac_client
import planetary_computer
from odc.stac import stac_load

import pandas as pd

In [ ]:
time_window = "2021-06-01/2021-08-01"
bounds = (lower_left[1], lower_left[0], upper_right[1], upper_right[0])

stac = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
search = stac.search(
    bbox=bounds,
    datetime=time_window,
    collections=["landsat-c2-l2"],
    query={"eo:cloud_cover": {"lt": 50},"platform": {"in": ["landsat-8"]}},
)

items = search.item_collection()

signed_items = [planetary_computer.sign(item).to_dict() for item in items]
resolution = 30
scale = resolution / 111320.0

data2 = stac_load(
    items,
    bands=["lwir11", "red", "nir08", "blue", "green", "swir22"],
    crs="EPSG:4326",
    resolution=scale,
    chunks={"x": 2048, "y": 2048},
    dtype="float64",
    patch_url=planetary_computer.sign,
    bbox=bounds
)

scale2 = 0.00341802
offset2 = 149.0
kelvin_celsius = 273.15

scale1 = 0.0000275
offset1 = -0.2


In [ ]:
data2 = data2.persist()

In [ ]:
final_ = data2.isel(time=2)

In [ ]:
lwir11 = final_["lwir11"]
nir08 = final_["nir08"]
red = final_["red"]
blue = final_["blue"]
green = final_["green"]
swir22 = final_["swir22"]

In [ ]:
red_ = red.astype(float) * scale1 + offset1
green_ = green.astype(float)*scale1 + offset1
blue_=blue.astype(float)*scale1 + offset1

In [ ]:
def pooling(data, n1, m1):
  pooled_da = data.rolling(latitude=n1, longitude=m1, center=True).mean()
  pooled_da = pooled_da.fillna(data)
  return pooled_da

R5 = pooling(red_, 10, 10)
G5 = pooling(green_, 10, 10)
B5 = pooling(blue_, 10, 10)

In [ ]:
import rioxarray as rxr
import rioxarray

def values(latitude , longitude , data, count ):
    try:
        value = data.sel(longitude =longitude, latitude =latitude, method="nearest").values.item()
        return value

    except (KeyError, ValueError):
        count[0] += 1
        return np.nan


In [ ]:
train_df = pd.read_csv(r'/content/train_df.csv')

In [ ]:
R5 = R5.persist()
G5 = G5.persist()
B5 = B5.persist()

In [ ]:
count = [0]
train_df["R10"] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], R5, count), axis=1)

In [ ]:
train_df["G10"] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], G5, count), axis=1)
train_df["B10"] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], B5, count), axis=1)

In [ ]:
train_df[["B10","G10", "R10", "UHI Index"]].corr()

,B10,G10,R10,UHI Index
B10,1.000000,0.993032,0.993822,0.359426
G10,0.993032,1.000000,0.995064,0.376785
R10,0.993822,0.995064,1.000000,0.371355
UHI Index,0.359426,0.376785,0.371355,1.000000


In [ ]:
red_ = red_.persist()
green_ = green_.persist()
blue_ = blue_.persist()
train_df['R'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], red_, count), axis=1)
train_df['G'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], green_, count), axis=1)
train_df['B'] =train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], blue_, count), axis=1)

In [ ]:
train_df[["R", "G", "B", "UHI Index"]].corr()

,R,G,B,UHI Index
R,1.000000,0.984704,0.959011,0.204945
G,0.984704,1.000000,0.978463,0.190245
B,0.959011,0.978463,1.000000,0.179773
UHI Index,0.204945,0.190245,0.179773,1.000000


In [ ]:
x1 = train_df['R10'] + train_df['G10'] + (train_df['B10'])

In [ ]:
x1.corr(train_df['UHI Index'])

np.float64(0.3702882646861839)

In [ ]:
# let's do avaerage of others
swir22 = final_["swir22"]
lwir11 = final_["lwir11"]
nir08 = final_["nir08"]

swir22_ = pooling(red_, 15 , 15)
lwir11_ = pooling(green_, 15, 15)
nir08_ = pooling(blue_, 15, 15)

swir22_ = swir22_.persist()
lwir11_ = lwir11_.persist()
nir08_ = nir08_.persist()
train_df['S'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], swir22_, count), axis=1)
train_df['L'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], lwir11_, count), axis=1)
train_df['N'] =train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], nir08_, count), axis=1)


In [ ]:
train_df[["S", "L", "N", "UHI Index"]].corr()

,S,L,N,UHI Index
S,1.000000,0.995585,0.995587,0.428990
L,0.995585,1.000000,0.993895,0.439626
N,0.995587,0.993895,1.000000,0.416245
UHI Index,0.428990,0.439626,0.416245,1.000000


In [ ]:
swir22 = final_["swir22"]
lwir11 = final_["lwir11"]
nir08 = final_["nir08"]

swir22_ = pooling(red_, 35 , 35)
lwir11_ = pooling(green_, 35, 35)
nir08_ = pooling(blue_, 35, 35)

swir22_ = swir22_.persist()
lwir11_ = lwir11_.persist()
nir08_ = nir08_.persist()
train_df['S35'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], swir22_, count), axis=1)
train_df['L35'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], lwir11_, count), axis=1)
train_df['N35'] =train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], nir08_, count), axis=1)

In [ ]:
ndvi_ = (nir08 - red_)/ (nir08 + red_)
nid_ = (red_ - blue_)/ (red_ + blue_)
ndwi_ = (green_ - nir08)/ (green_ + nir08)

ndvi_ = ndvi_.persist()
nid_ = nid_.persist()
ndwi_ = ndwi_.persist()

In [ ]:

ndvi = pooling(ndvi_, 55 , 55)
nid = pooling(nid_, 55, 55)
ndwi = pooling(ndwi_, 55, 55)

ndvi = ndvi.persist()
nid = nid.persist()
ndwi = ndwi.persist()

train_df['ndvi55'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], ndvi, count), axis=1)
train_df['nid55'] = train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], nid, count), axis=1)
train_df['ndwi55'] =train_df.apply(lambda row: values(row['Latitude'], row['Longitude'], ndwi, count), axis=1)

In [ ]:
train_df[["ndvi55", "nid55", "ndwi55", "UHI Index"]].corr()

,ndvi55,nid55,ndwi55,UHI Index
ndvi55,1.000000,-0.754005,-0.995214,-0.440424
nid55,-0.754005,1.000000,0.730956,0.510552
ndwi55,-0.995214,0.730956,1.000000,0.435943
UHI Index,-0.440424,0.510552,0.435943,1.000000


In [ ]:
# nid45, ndvi45, L35, N,

In [ ]:
trainD = train_df[["N", "nid45", "ndvi55","L35", "UHI Index"]]

In [ ]:
trainD.drop_duplicates(inplace=True)

In [ ]:

import numpy as np
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

X = trainD.drop('UHI Index', axis=1)
y = trainD['UHI Index']

n_splits = 5
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

rmse_scores = []

# Iterate through the folds
for fold, (train_index, val_index) in enumerate(kf.split(X)):
    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    model = XGBRegressor(learning_rate=0.001, n_estimators=20000, random_state=42)  # Use a low learning rate
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    r2 = model.score(X_val, y_val)
    print(r2)

print(f"Average RMSE across {n_splits} folds: {np.mean(r2)}")


0.8018859740405743
0.7608671223037511
0.759789716791843
0.7616267049002876
0.7773570770442021
Average RMSE across 5 folds: 0.7773570770442021


In [ ]:
#  train_df[["N", "nid45", "ndvi55","L35", "UHI Index"]]
subm